In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import json
import regex
import unidecode

In [2]:
# Load data
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/insiders.parquet")
df_in = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/outsiders.parquet")
df_ou = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/minors.parquet")
df_mi = pd.read_parquet(dir_df)

In [3]:
print(df_in.shape)
print(df_ou.shape)
print(df_mi.shape)

(519520, 195)
(236210, 43)
(1650869, 107)


In [4]:
# Load info from gencat
tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")

C:\Users\josea\AppData\Local\Temp\ipykernel_404\694994485.py:2: DtypeWarning: Columns (27,43,52,53,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  tend_cat = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")


In [5]:
# with open(r"C:\Users\josea\Downloads\genCat_Junio_2023.json", "r") as f:
#     gencat = pd.json_normalize(json.load(f))

# Aux functions

In [6]:
def unify_colname(col):
    return ".".join([el for el in col if el])


def evaluate(el):
    if not isinstance(el, str):
        return el
    if el[0] == "[" or el[0] == "'":
        return eval(el)
    return el


def fill_na(cell, fill=[]):
    """
    Fill elements in pd.DataFrame with `fill`.
    """
    if hasattr(cell, "__iter__"):
        if isinstance(cell, str):
            if cell == "nan":
                return fill
            return cell
        nas = []
        for el in cell:
            if el == "nan" or pd.isna(el):
                nas.append(True)
            else:
                nas.append(False)
        if all(nas):
            return fill
        return cell
    if pd.isna(cell):
        return fill
    return cell


def process_str(el: str):
    s = regex.sub(r"(^[\W]*)|([\W]*$)|([^\w:/-\s])", "", el).lower().strip()
    s = regex.sub(r"\s+", " ", s)
    return s


def process_cpv(el: str):
    return regex.sub(r"\D", "", el)

In [7]:
# Get general tender info (title, object, winner, etc)
df_in.columns = [unify_colname(c) for c in df_in.columns]
df_ou.columns = [unify_colname(c) for c in df_ou.columns]
df_mi.columns = [unify_colname(c) for c in df_mi.columns]

In [8]:
use_cols = [
    "id",
    "summary",
    "title",
    "updated",
    # "deleted_on",
    "ContractFolderStatus.ContractFolderID",
    "ContractFolderStatus.ContractFolderStatusCode",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyIdentification.ID",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyName.Name",
    "ContractFolderStatus.ProcurementProject.Name",
    "ContractFolderStatus.ProcurementProject.TypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TaxExclusiveAmount",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.DurationMeasure",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.StartDate",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.EndDate",
    # "ContractFolderStatus.TenderResult.ResultCode",
    # "ContractFolderStatus.TenderResult.ReceivedTenderQuantity",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
    "ContractFolderStatus.TenderResult.WinningParty.PartyName.Name",
    # "ContractFolderStatus.TenderResult.AwardedTenderedProject.LegalMonetaryTotal.TaxExclusiveAmount",
    # "ContractFolderStatus.TenderingProcess.ProcedureCode",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndDate",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndTime",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.Description",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod",
    # "ContractFolderStatus.ProcurementProject.SubTypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TotalAmount",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.IdentificationCode",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.Name",
    # "ContractFolderStatus.TenderResult.Description",
    # "ContractFolderStatus.TenderResult.AwardDate",
    # "ContractFolderStatus.TenderResult.StartDate",
    # "ContractFolderStatus.TenderResult.Contract.ID",
    # "ContractFolderStatus.TenderResult.Contract.IssueDate",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.CityName",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.PostalZone",
    "ContractFolderStatus.TenderingTerms.FundingProgramCode",
    "ContractFolderStatus.TenderingTerms.FundingProgram",
    "ContractFolderStatus.TenderResult.WinningParty.PartyLegalEntity.CompanyTypeCode",
]

In [9]:
rename_columns = {
    "title": "title",
    "summary": "summary",
    "ContractFolderStatus.ContractFolderID": "id",
    "ContractFolderStatus.ProcurementProject.Name": "project",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID": "winner",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity": "location",
    "denominacio": "title",
    "objecte_contracte": "summary",
    "codi_expedient": "id",
    "codi_cpv": "cpv",
}

# PLACE

In [10]:
df = pd.concat(
    [
        df_in[[c for c in use_cols if c in df_in.columns]],
        df_ou[[c for c in use_cols if c in df_ou.columns]],
        # df_mi[[c for c in use_cols if c in df_mi.columns]],
    ]
)
index_names = df.index.names
df.reset_index(inplace=True)
df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
# df.drop(index_names, inplace=True, axis=1)
df.set_index("identifier", inplace=True)
df = df.applymap(fill_na, fill=np.nan)
df.shape

(755730, 24)

### Get relevant data

In [11]:
# Get only relevant data
tender_filt = (
    df[
        [
            "title",
            "summary",
            "ContractFolderStatus.ContractFolderID",
            # "ContractFolderStatus.ProcurementProject.Name",
            # "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
            "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
            "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
        ]
    ]
    .rename(columns=rename_columns)
    .dropna(how="all")
    .explode("cpv")
    # .explode("winner")
)
tender_filt["id_orig"] = tender_filt["id"]
tender_filt["id"] = tender_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
# tender_filt["winner"] = tender_filt["winner"].apply(evaluate)
tender_filt["cpv"] = tender_filt["cpv"].apply(evaluate)
tender_filt = tender_filt.explode("cpv")  # .explode("winner")
tender_filt["cpv"] = tender_filt["cpv"].astype(str).apply(process_cpv)
tender_filt["location"] = tender_filt["location"].apply(
    lambda x: unidecode.unidecode(x.lower()) if not pd.isna(x) else np.nan
)
tender_filt = (
    tender_filt.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
)
tender_filt["cpv_div"] = tender_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)
tender_filt = tender_filt.reset_index().reset_index()

### Get data from cat

In [63]:
# Do the same, but in this case using the original id (id_orig)
agg_tender_orig = (
    tender_filt[
        tender_filt["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .drop("identifier", axis=1)
    .groupby(["id_orig"])
    .agg(list)
)
valid_agg_tender_orig = agg_tender_orig[
    agg_tender_orig["title"].apply(lambda x: len(set(x)) == 1)
]
valid_agg_tender_orig[valid_agg_tender_orig.columns[1:]] = valid_agg_tender_orig[
    valid_agg_tender_orig.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_agg_tender_orig = valid_agg_tender_orig.reset_index()
print(valid_agg_tender_orig.shape)
display(valid_agg_tender_orig.head())

(252104, 8)


C:\Users\josea\AppData\Local\Temp\ipykernel_404\160396498.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_agg_tender_orig[valid_agg_tender_orig.columns[1:]] = valid_agg_tender_orig[


,id_orig,index,title,summary,id,cpv,location,cpv_div
0,0/2020-5/2020,[990577],rehabilitació de casa de poblet,id licitación: 0/2020-5/2020 órgano de contrat...,0-2020-5-2020,454541000,nan,45
1,00/0000,[884512],licitació de prova sobre digital 00/0000 - pro...,id licitación: 00/0000 órgano de contratación:...,00-0000,791000000,nan,79
2,00/2019,[843883],comunicació de contractació anual programada e...,id licitación: 00/2019 órgano de contratación:...,00-2019,853200000,nan,85
3,00/d/12/0000572,[996222],servicio de mantenimiento del sistema de alime...,id licitación: 00/d/12/0000572 órgano de contr...,00-d-12-0000572,505320000,nan,50
4,00/d/12/421,"[849661, 849662]",servicio de mantenimiento de la licencia dynat...,id licitación: 00/d/12/421 órgano de contratac...,00-d-12-421,48700000,nan,48


In [64]:
# Filter using only those from Cat
agg_tender = (
    tender_filt[
        tender_filt["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .drop("identifier", axis=1)
    .groupby(["id"])
    .agg(list)
)
valid_agg_tender = agg_tender[agg_tender["title"].apply(lambda x: len(set(x)) == 1)]
valid_agg_tender[valid_agg_tender.columns[1:]] = valid_agg_tender[
    valid_agg_tender.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_agg_tender = valid_agg_tender.reset_index()
print(valid_agg_tender.shape)
display(valid_agg_tender.head())

C:\Users\josea\AppData\Local\Temp\ipykernel_404\2700280501.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_agg_tender[valid_agg_tender.columns[1:]] = valid_agg_tender[


(245975, 8)


,id,index,title,summary,cpv,location,id_orig,cpv_div
0,0-2020-5-2020,[990577],rehabilitació de casa de poblet,id licitación: 0/2020-5/2020 órgano de contrat...,454541000,nan,0/2020-5/2020,45
1,00-0000,[884512],licitació de prova sobre digital 00/0000 - pro...,id licitación: 00/0000 órgano de contratación:...,791000000,nan,00/0000,79
2,00-2019,[843883],comunicació de contractació anual programada e...,id licitación: 00/2019 órgano de contratación:...,853200000,nan,00/2019,85
3,00-d-12-0000572,[996222],servicio de mantenimiento del sistema de alime...,id licitación: 00/d/12/0000572 órgano de contr...,505320000,nan,00/d/12/0000572,50
4,00-d-12-421,"[849661, 849662]",servicio de mantenimiento de la licencia dynat...,id licitación: 00/d/12/421 órgano de contratac...,48700000,nan,00/d/12/421,48


In [14]:
# a = tend_cat["codi_expedient"]
# a[a.apply(lambda x: bool(regex.search(r"\s{2,}", x)) if not pd.isna(x) else False)]

# GENCAT

In [65]:
# Clean tenders cat
tend_cat_filt = (
    tend_cat[
        [
            "denominacio",
            "objecte_contracte",
            "codi_expedient",
            "codi_cpv",
        ]
    ]
    .rename(columns=rename_columns)
    .astype(str)
    .applymap(process_str)
)
tend_cat_filt["id_orig"] = tend_cat_filt["id"]
tend_cat_filt["id"] = tend_cat_filt["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
tend_cat_filt["cpv"] = tend_cat_filt["cpv"].astype(str).apply(process_cpv)
tend_cat_filt = (
    tend_cat_filt.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
)
tend_cat_filt["cpv_div"] = tend_cat_filt["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)
tend_cat_filt = tend_cat_filt.reset_index()

In [17]:
# tend_cat_filt[tend_cat_filt["id"]=="ccs-2022-6"]
# c = []
# for el in tend_cat[:3].apply(lambda x: Counter(x), axis=0):
#     c.append(3 not in el.values())
# tend_cat[:3][tend_cat.columns[c]]

In [69]:
# Aggregate by original ID
valid_tend_cat_orig = tend_cat_filt.groupby("id_orig").agg(list)
valid_tend_cat_orig = valid_tend_cat_orig[
    valid_tend_cat_orig["title"].apply(lambda x: len(set(x)) == 1)
]
valid_tend_cat_orig[valid_tend_cat_orig.columns[1:]] = valid_tend_cat_orig[
    valid_tend_cat_orig.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_tend_cat_orig = valid_tend_cat_orig.reset_index()
print(len(valid_tend_cat_orig))
display(valid_tend_cat_orig.head())

232112


,id_orig,index,title,summary,id,cpv,cpv_div
0,0-s-04/2023,[127873],gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,0-s-04-2023,794210001,79
1,0/2020-5/2020,[153086],rehabilitació de casa de poblet,rehabilitació de casa de poblet,0-2020-5-2020,454541005,45
2,00/0000,[283367],licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,00-0000,791000005,79
3,000,[272105],concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,000,NaN,NaN
4,000 expedient 2020 034,[182156],000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,000-expedient-2020-034,926220007,92


In [70]:
# Aggregate by ID
valid_tend_cat = tend_cat_filt.groupby("id").agg(list)
valid_tend_cat = valid_tend_cat[
    valid_tend_cat["title"].apply(lambda x: len(set(x)) == 1)
]
valid_tend_cat[valid_tend_cat.columns[1:]] = valid_tend_cat[
    valid_tend_cat.columns[1:]
].applymap(lambda x: Counter(x).most_common()[0][0])
valid_tend_cat = valid_tend_cat.reset_index()
print(len(valid_tend_cat))
display(valid_tend_cat.head())

227651


,id,index,title,summary,cpv,id_orig,cpv_div
0,0-2020-5-2020,[153086],rehabilitació de casa de poblet,rehabilitació de casa de poblet,454541005,0/2020-5/2020,45
1,0-s-04-2023,[127873],gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,794210001,0-s-04/2023,79
2,00-0000,[283367],licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,791000005,00/0000,79
3,000,[272105],concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,NaN,000,NaN
4,000-expedient-2020-034,[182156],000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,926220007,000 expedient 2020 034,92


# TODO
- Hacer match con budget

# Match PLACE & GENCAT

### By ID

In [77]:
# Match tenders by ID
matched_id_orig = pd.merge(
    valid_agg_tender_orig,
    valid_tend_cat_orig,
    how="inner",
    left_on=["id_orig"],
    right_on=["id_orig"],
)
matched_id_orig = matched_id_orig[
    [
        "id_orig",
        "index_x",
        "index_y",
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
    ]
]
title = []
cpv_div = []
for tx, ty, cx, cy in matched_id_orig[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    title.append(Counter(set([tx, ty])))
    cpv_div.append(Counter(set([cx, cy])))

matched_id_orig["title"] = title
matched_id_orig["cpv_div"] = cpv_div
matched_id_orig = matched_id_orig.rename(
    columns={
        "index_x": "index_place",
        "index_y": "index_cat",
    }
)

matched_id_orig = matched_id_orig[
    ["id_orig", "index_place", "index_cat", "title", "cpv_div"]
]
print(len(matched_id_orig))
print(sum(matched_id_orig["title"].apply(lambda x: sum(x.values()))))
display(matched_id_orig.head())

91792
154045


,id_orig,index_place,index_cat,title,cpv_div
0,0/2020-5/2020,[990577],[153086],{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00/0000,[884512],[283367],{'licitació de prova sobre digital 00/0000 - p...,{'79': 1}
2,000 expedient 2020 034,[903809],[182156],{'en el marc duna creixent proliferació quant ...,{'92': 1}
3,000 expedient 2021 266,[972632],[157778],{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057/2022,[1007173],[189410],{'renovació de la xarxa daigua potable i impla...,{'45': 1}


In [78]:
# Match tenders by ID
matched_id = pd.merge(
    valid_agg_tender,
    valid_tend_cat,
    how="inner",
    left_on=["id"],
    right_on=["id"],
)
matched_id = matched_id[
    [
        "id",
        "index_x",
        "index_y",
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
    ]
]
title = []
cpv_div = []
for tx, ty, cx, cy in matched_id[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    title.append(Counter(set([tx, ty])))
    cpv_div.append(Counter(set([cx, cy])))

matched_id["title"] = title
matched_id["cpv_div"] = cpv_div
matched_id = matched_id.rename(
    columns={
        "index_x": "index_place",
        "index_y": "index_cat",
    }
)

matched_id = matched_id[["id", "index_place", "index_cat", "title", "cpv_div"]]
print(len(matched_id))
print(sum(matched_id["title"].apply(lambda x: sum(x.values()))))
display(matched_id.head())

83775
140485


,id,index_place,index_cat,title,cpv_div
0,0-2020-5-2020,[990577],[153086],{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,[884512],[283367],{'licitació de prova sobre digital 00/0000 - p...,{'79': 1}
2,000-expedient-2020-034,[903809],[182156],{'en el marc duna creixent proliferació quant ...,{'92': 1}
3,000-expedient-2021-266,[972632],[157778],{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,[1007173],[189410],{'renovació de la xarxa daigua potable i impla...,{'45': 1}


In [146]:
matched_id.head()

,id,index_place,index_cat,title,cpv_div
0,0-2020-5-2020,[990577],[153086],{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,[884512],[283367],{'licitació de prova sobre digital 00/0000 - p...,{'79': 1}
2,000-expedient-2020-034,[903809],[182156],{'en el marc duna creixent proliferació quant ...,{'92': 1}
3,000-expedient-2021-266,[972632],[157778],{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,[1007173],[189410],{'renovació de la xarxa daigua potable i impla...,{'45': 1}


In [147]:
matched_id_orig.head()

,id_orig,index_place,index_cat,title,cpv_div
0,0/2020-5/2020,[990577],[153086],{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00/0000,[884512],[283367],{'licitació de prova sobre digital 00/0000 - p...,{'79': 1}
2,000 expedient 2020 034,[903809],[182156],{'en el marc duna creixent proliferació quant ...,{'92': 1}
3,000 expedient 2021 266,[972632],[157778],{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057/2022,[1007173],[189410],{'renovació de la xarxa daigua potable i impla...,{'45': 1}


In [184]:
# Indices from PLACE that have been matched by ID
place2m_id = {
    k: v
    for v, k in matched_id["index_place"].reset_index().explode("index_place").values
}
place2m_id_orig = {
    k: v
    for v, k in matched_id_orig["index_place"]
    .reset_index()
    .explode("index_place")
    .values
}
print(f"{'Matched from PLACE by ID: ':<40}{len(place2m_id)}")
print(f"{'Matched from PLACE by original ID: ':<40}{len(place2m_id_orig)}")
print(f"{'Difference: ':<40}{abs(len(place2m_id)-len(place2m_id_orig))}")
print("But actually, the indices that are unique to any of them is")
print(f"{'Unique (difference between orig and processed): ':<40}{len(place2m_id.keys() ^ place2m_id_orig.keys())}")
print(f"{'Matched only by original ID: ':<40}{len(place2m_id_orig.keys()-place2m_id.keys())}")
print(f"{'Matched only by processed ID: ':<40}{len(place2m_id.keys()-place2m_id_orig.keys())}")

Matched from PLACE by ID:               89500
Matched from PLACE by original ID:      97968
Difference:                             8468
But actually, the indices that are unique to any of them is
Unique (difference between orig and processed): 8826
Matched only by original ID:            8647
Matched only by processed ID:           179


In [186]:
# Indices from PLACE that have been matched by ID
cat2m_id = {
    k: v
    for v, k in matched_id["index_cat"].reset_index().explode("index_cat").values
}
cat2m_id_orig = {
    k: v
    for v, k in matched_id_orig["index_cat"]
    .reset_index()
    .explode("index_cat")
    .values
}
print(f"{'Matched from GENCAT by ID: ':<40}{len(cat2m_id)}")
print(f"{'Matched from GENCAT by original ID: ':<40}{len(cat2m_id_orig)}")
print(f"{'Difference: ':<40}{abs(len(cat2m_id)-len(cat2m_id_orig))}")
print("But actually, the indices that are unique to any of them is")
print(f"{'Unique (difference between orig and processed): ':<40}{len(cat2m_id.keys() ^ cat2m_id_orig.keys())}")
print(f"{'Matched only by original ID: ':<40}{len(cat2m_id_orig.keys()-cat2m_id.keys())}")
print(f"{'Matched only by processed ID: ':<40}{len(cat2m_id.keys()-cat2m_id_orig.keys())}")

Matched from GENCAT by ID:              111237
Matched from GENCAT by original ID:     121317
Difference:                             10080
But actually, the indices that are unique to any of them is
Unique (difference between orig and processed): 10536
Matched only by original ID:            10308
Matched only by processed ID:           228


### By title and CPV

In [119]:
# Match tenders by title and CPV
matched_aux = pd.merge(
    valid_agg_tender,
    valid_tend_cat,
    how="inner",
    left_on=[
        "title",
        "cpv_div",
    ],
    right_on=[
        "title",
        "cpv_div",
    ],
)
matched_aux = matched_aux[
    [
        "id_x",
        "id_y",
        "id_orig_x",
        "id_orig_y",
        "index_x",
        "index_y",
        "title",
        "cpv_div",
    ]
]
matched_aux = matched_aux[matched_aux["id_x"] != matched_aux["id_y"]]

# # ERROR
# matched_aux[
#     (~matched_aux["id_x"].isin(matched_id["id"]))
#     & (~matched_aux["id_y"].isin(matched_id["id"]))
# ]

# Split unique id_x and id_y
m_x = matched_aux.loc[
    ~matched_aux["id_x"].isin(matched_id["id"]),
    ["id_x", "id_orig_x", "index_x", "title", "cpv_div"],
].rename({"id_x": "id", "id_orig_x": "id_orig"}, axis=1)
m_y = matched_aux.loc[
    ~matched_aux["id_y"].isin(matched_id["id"]),
    ["id_y", "id_orig_y", "index_y", "title", "cpv_div"],
].rename({"id_y": "id", "id_orig_y": "id_orig"}, axis=1)
# matched_aux = pd.concat([m_x, m_y]).groupby("id").agg(Counter).reset_index()

In [130]:
# ID orig
matched_aux_orig = (
    pd.concat([m_x, m_y])
    .groupby("id_orig")
    .agg(
        {
            "id": set,
            "title": Counter,
            "cpv_div": Counter,
            "index_x": sum,
            "index_y": sum,
        }
    )
    .reset_index()
).rename(
    columns={
        "index_x": "index_place",
        "index_y": "index_cat",
    }
)
matched_aux_orig["index_place"] = matched_aux_orig["index_place"].apply(
    lambda x: list(set(x)) if x else None
)
matched_aux_orig["index_cat"] = matched_aux_orig["index_cat"].apply(
    lambda x: list(set(x)) if x else None
)

matched_aux_orig = matched_aux_orig[
    ["id_orig", "id", "index_place", "index_cat", "title", "cpv_div"]
]
print(len(matched_aux_orig))
print(sum(matched_aux_orig["title"].apply(lambda x: sum(x.values()))))
display(matched_aux_orig.head())

1743
9836


,id_orig,id,index_place,index_cat,title,cpv_div
0,001/22,{001-22},None,[212400],{'contractació dun servei que inclogui 3 perso...,{'72': 1}
1,002/20,{002-20},None,[161093],{'contractació dun servei que inclogui 3 perso...,{'72': 1}
2,003/23,{003-23},[1069467],None,{'contractació dun servei que inclogui 3 perso...,{'72': 2}
3,0047,{0047},None,[284358],{'servei de manteniment preventiu i correctiu ...,{'50': 1}
4,006/19,{006-19},None,[194335],{'maquetació de la revista informat versió en ...,{'79': 1}


In [131]:
# ID
matched_aux = (
    pd.concat([m_x, m_y])
    .groupby("id")
    .agg(
        {
            "id_orig": set,
            "title": Counter,
            "cpv_div": Counter,
            "index_x": sum,
            "index_y": sum,
        }
    )
    .reset_index()
).rename(
    columns={
        "index_x": "index_place",
        "index_y": "index_cat",
    }
)
matched_aux["index_place"] = matched_aux["index_place"].apply(
    lambda x: list(set(x)) if x else None
)
matched_aux["index_cat"] = matched_aux["index_cat"].apply(
    lambda x: list(set(x)) if x else None
)

matched_aux = matched_aux[
    ["id", "id_orig", "index_place", "index_cat", "title", "cpv_div"]
]
print(len(matched_aux))
print(sum(matched_aux["title"].apply(lambda x: sum(x.values()))))
display(matched_aux.head())

2973
9836


,id,id_orig,index_place,index_cat,title,cpv_div
0,001-22,{001/22},None,[212400],{'contractació dun servei que inclogui 3 perso...,{'72': 1}
1,002-20,{002/20},None,[161093],{'contractació dun servei que inclogui 3 perso...,{'72': 1}
2,003-23,{003/23},[1069467],None,{'contractació dun servei que inclogui 3 perso...,{'72': 2}
3,0047,{0047},None,[284358],{'servei de manteniment preventiu i correctiu ...,{'50': 1}
4,006-19,{006/19},None,[194335],{'maquetació de la revista informat versió en ...,{'79': 1}


In [139]:
matched_aux_orig[matched_aux_orig["id"].apply(len)>1]

,id_orig,id,index_place,index_cat,title,cpv_div
5,0101619364,"{010-1619-364, 0101619364}",[836338],[215133],{'obres relatives al projecte executiu de lade...,{'45': 2}
6,0101619408,"{0101619408, 010-1619-408}",[868946],[289651],{'concurs de rehabilitació integral de ledific...,{'71': 2}
7,010407-14/18,"{01-04-07-14-18, 010407-14-18}",[835525],[150461],{'disseny muntatge i execució de lactivitat de...,{'79': 2}
8,010407-17-2018,"{01-04-07-17-2018, 010407-17-2018}",[846534],[184018],{'contractació del servei dactualització tecno...,{'48': 2}
9,010407-18/2018,"{01-04-07-18-2018, 010407-18-2018}",[839153],[211313],{'programa dacollida per a nous ciutadans': 2},{'98': 2}
...,...,...,...,...,...,...
1732,tnc201900032,"{tnc201900032, tnc-2019-00032}",[851521],[164602],{'construcció escenogràfica de lespectacle el ...,{'45': 2}
1734,tnc201900040,"{tnc-2019-00040, tnc201900040}",[840411],[10784],{'contractació de lescenògraf max glaenzel per...,{'92': 2}
1735,tnc201900041,"{tnc201900041, tnc-2019-00041}",[842521],[212882],{'contractació de la directora descena per lob...,{'92': 2}
1736,tnc201900042,"{tnc-2019-00042, tnc201900042}",[846892],[269302],{'contractació de lescenògraf enric planas ter...,{'92': 2}


### All (id, name, cpv)

In [133]:
pd.merge(
    matched_id,
    matched_aux,
    how="outer",
    left_on="id",
    right_on="id",
)

,id,index_place_x,index_cat_x,title_x,cpv_div_x,id_orig,index_place_y,index_cat_y,title_y,cpv_div_y
0,0-2020-5-2020,[990577],[153086],{'rehabilitació de casa de poblet': 1},{'45': 1},NaN,NaN,NaN,NaN,NaN
1,00-0000,[884512],[283367],{'licitació de prova sobre digital 00/0000 - p...,{'79': 1},NaN,NaN,NaN,NaN,NaN
2,000-expedient-2020-034,[903809],[182156],{'en el marc duna creixent proliferació quant ...,{'92': 1},NaN,NaN,NaN,NaN,NaN
3,000-expedient-2021-266,[972632],[157778],{'000 expedient 2021 266 - expedient de contra...,{'34': 1},NaN,NaN,NaN,NaN,NaN
4,00000057-2022,[1007173],[189410],{'renovació de la xarxa daigua potable i impla...,{'45': 1},NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
86743,uvic-ucc-201704,NaN,NaN,NaN,NaN,{uvic-ucc-201704},None,[196237],{'servei postal dels diversos centres de la un...,{'60': 1}
86744,x2021000479,NaN,NaN,NaN,NaN,{x2021000479},None,[51138],{'subministrament de material informàtic': 2},{'30': 2}
86745,x2023000113,NaN,NaN,NaN,NaN,{x2023000113},[1072623],None,{'servei de recollida danimals de companyia ab...,{'98': 2}
86746,x2023000547,NaN,NaN,NaN,NaN,{x2023000547},None,[47425],{'contracte de serveis de neteja dels edificis...,{'90': 1}


In [142]:
pd.merge(
    matched_id_orig,
    matched_aux_orig,
    how="outer",
    left_on="id_orig",
    right_on="id_orig",
)

,id_orig,index_place_x,index_cat_x,title_x,cpv_div_x,id,index_place_y,index_cat_y,title_y,cpv_div_y
0,0/2020-5/2020,[990577],[153086],{'rehabilitació de casa de poblet': 1},{'45': 1},NaN,NaN,NaN,NaN,NaN
1,00/0000,[884512],[283367],{'licitació de prova sobre digital 00/0000 - p...,{'79': 1},NaN,NaN,NaN,NaN,NaN
2,000 expedient 2020 034,[903809],[182156],{'en el marc duna creixent proliferació quant ...,{'92': 1},NaN,NaN,NaN,NaN,NaN
3,000 expedient 2021 266,[972632],[157778],{'000 expedient 2021 266 - expedient de contra...,{'34': 1},NaN,NaN,NaN,NaN,NaN
4,00000057/2022,[1007173],[189410],{'renovació de la xarxa daigua potable i impla...,{'45': 1},NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
92244,uvic-ucc-201704,NaN,NaN,NaN,NaN,{uvic-ucc-201704},None,[196237],{'servei postal dels diversos centres de la un...,{'60': 1}
92245,x2021000479,NaN,NaN,NaN,NaN,{x2021000479},None,[51138],{'subministrament de material informàtic': 2},{'30': 2}
92246,x2023000113,NaN,NaN,NaN,NaN,{x2023000113},[1072623],None,{'servei de recollida danimals de companyia ab...,{'98': 2}
92247,x2023000547,NaN,NaN,NaN,NaN,{x2023000547},None,[47425],{'contracte de serveis de neteja dels edificis...,{'90': 1}


In [ ]:
# Combine both
matched_final_orig = pd.merge(
    matched_id_orig,
    matched_aux_orig,
    how="outer",
    left_on="id_orig",
    right_on="id_orig",
)
title = []
cpv_div = []
for tx, ty, cx, cy in matched_final[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    t = dict()
    c = dict()
    t.update(tx if not pd.isna(tx) else dict())
    t.update(ty if not pd.isna(ty) else dict())
    c.update(cx if not pd.isna(cx) else dict())
    c.update(cy if not pd.isna(cy) else dict())
    title.append(t)
    cpv_div.append(c)

matched_final["title"] = title
matched_final["cpv_div"] = cpv_div

matched_final = matched_final[["id", "title", "cpv_div"]]
print(len(matched_final))
print(sum(matched_final["title"].apply(lambda x: sum(x.values()))))
display(matched_final.head())

In [42]:
# Combine both
matched_final = pd.merge(
    matched_id,
    matched_aux,
    how="outer",
    left_on="id",
    right_on="id",
)
title = []
cpv_div = []
for tx, ty, cx, cy in matched_final[
    ["title_x", "title_y", "cpv_div_x", "cpv_div_y"]
].values:
    t = dict()
    c = dict()
    t.update(tx if not pd.isna(tx) else dict())
    t.update(ty if not pd.isna(ty) else dict())
    c.update(cx if not pd.isna(cx) else dict())
    c.update(cy if not pd.isna(cy) else dict())
    title.append(t)
    cpv_div.append(c)

matched_final["title"] = title
matched_final["cpv_div"] = cpv_div

matched_final = matched_final[["id", "title", "cpv_div"]]
print(len(matched_final))
print(sum(matched_final["title"].apply(lambda x: sum(x.values()))))
display(matched_final.head())

86748
150321


,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,{'licitació de prova sobre digital 00/0000 - p...,{'79': 1}
2,000-expedient-2020-034,{'en el marc duna creixent proliferació quant ...,{'92': 1}
3,000-expedient-2021-266,{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa de subministrament dai...,{'45': 1}


In [53]:
# Unique indices (by id)
_tot_id = len(matched_id) / len(valid_tend_cat)
# Total elements (taking into account some values appear multiple times)
_tot_id_sum = matched_id["title"].apply(lambda x: sum(x.values())).sum() / len(
    valid_tend_cat
)

# Unique indices (by cpv and title)
_tot_ti = len(matched_aux) / len(valid_tend_cat)
# Total elements (taking into account some values appear multiple times)
_tot_ti_sum = matched_aux["title"].apply(lambda x: sum(x.values())).sum() / len(
    valid_tend_cat
)

# Unique aggregated
_tot = len(matched_final) / len(valid_tend_cat)
# Total elements (taking into account some values appear multiple times)
_tot_sum = matched_final["title"].apply(lambda x: sum(x.values())).sum() / len(
    valid_tend_cat
)

print(f"{'Unique indices (by id): ':<40}{_tot_id:.5f}")
print(f"{'Total elements (by id): ':<40}{_tot_id_sum:.5f}")
print()
print(f"{'Unique indices (by cpv and title): ':<40}{_tot_ti:.5f}")
print(f"{'Total elements (by cpv and title): ':<40}{_tot_ti_sum:.5f}")
print()
print(f"{'Unique indices (aggregated): ':<40}{_tot:.5f}")
print(f"{'Total elements (aggregated): ':<40}{_tot_sum:.5f}")

Unique indices (by id):                 0.36800
Total elements (by id):                 0.61711

Unique indices (by cpv and title):      0.01306
Total elements (by cpv and title):      0.04321

Unique indices (aggregated):            0.38106
Total elements (aggregated):            0.66031
